# Part 3: Data Analytics

## Step 1: Selecting real-world dataset

The dataset that will be used today are the stock market data of TESLA and some companies related to it. It is extracted using yfinance API. yfinance is an open source library developed by Ran Aroussi as a means to access the financial data available on Yahoo Finance[1]. Out of the many variables extracted, we will be focusing on Closing Price and Volume of shares traded. 

In [ ]:
#installing needed libraries
pip install yfinance

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
# Importing packages
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Dropout

The stock market data is extracted by creating a ticker object for the particular stocks we need data for. The primary company I am interested is TESLA (TSLA). They are growing at a tremendous pace and is currently the biggest electric vehicle manufacturer in the world [2]. I am also interested to know the effect of TSLA share price have on its main battery supplier, Panasonic (PCRFY) [3]. I also intend to find the effect of TSLA share price on its competitor in the field of vehicle manufacturing (Ford(F) and General Motors(GM)) and energy (Royal Dutch Shell(RDS-B) and BP(BP)). We will be extracting data of last 5 years.

In [ ]:
# Setting the start and end date
start_date = '2016-11-30'
end_date = '2021-12-01'

# Set the ticker
ticker = 'TSLA','PCRFY','GM','F','RDS-B','BP'

# Get the data
data = yf.download(ticker, start_date, end_date,group_by='tickers')

# Print data
data

In [ ]:
# Saving the data into a csv file
data.to_csv('data.csv')

## Step 2: Data preparation and Cleaning

First step is we will be checking there are any null values in the dataset. We will also create 3 additional columns for the variables Percentage change of Price(%changeprice), Percentage change of volume(%changevolume) and Cumulative Return of the stock(CumulativeRet).

In [ ]:
# Loading the dataset into a data fram
stock_data = pd.read_csv('data.csv', header=[0, 1], index_col=0)
# Print dataset
stock_data

In [ ]:
# Print the shape of dataset
stock_data.shape

In [ ]:
# Checking no. of null values
stock_data.isnull().sum()

In [ ]:
# Checking any numeric data is not string
stock_data.dtypes.value_counts()

###### Pre-Processing Completed

We can see that there is no Null value in the data, and also all the stock values are not strings either. Both of these are very good indicators that all values are present.

###### Creation of additional columns

In [ ]:
# Calculating daily change of price
tesladailychanges= stock_data[( 'TSLA', 'Adj Close')]
teslapercentagedailychange= tesladailychanges.pct_change(periods=1)
stock_data['TSLA','%changeprice'] = teslapercentagedailychange

panasonicdailychanges= stock_data[( 'PCRFY', 'Adj Close')]
panasonicpercentagedailychange= panasonicdailychanges.pct_change(periods=1)
stock_data['PCRFY','%changeprice'] = panasonicpercentagedailychange

GMdailychanges= stock_data[( 'GM', 'Adj Close')]
GMpercentagedailychange= GMdailychanges.pct_change(periods=1)
stock_data['GM','%changeprice'] = GMpercentagedailychange

forddailychanges= stock_data[( 'F', 'Adj Close')]
fordpercentagedailychange= forddailychanges.pct_change(periods=1)
stock_data['F','%changeprice'] = fordpercentagedailychange

shelldailychanges= stock_data[( 'RDS-B', 'Adj Close')]
shellpercentagedailychange= shelldailychanges.pct_change(periods=1)
stock_data['RDS-B','%changeprice'] = shellpercentagedailychange

bpdailychanges= stock_data[( 'BP', 'Adj Close')]
bppercentagedailychange= bpdailychanges.pct_change(periods=1)
stock_data['BP','%changeprice'] = bppercentagedailychange

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Calculating daily change of volume traded
teslavolumechanges= stock_data[( 'TSLA', 'Volume')]
teslapercentagevolumechange= teslavolumechanges.pct_change(periods=1)
stock_data['TSLA','%changevolume'] = teslapercentagevolumechange

panasonicvolumechanges= stock_data[( 'PCRFY', 'Volume')]
panasonicpercentagevolumechange= panasonicvolumechanges.pct_change(periods=1)
stock_data['PCRFY','%changevolume'] = panasonicpercentagevolumechange

GMvolumechanges= stock_data[( 'GM', 'Volume')]
GMpercentagevolumechange= GMvolumechanges.pct_change(periods=1)
stock_data['GM','%changevolume'] = GMpercentagevolumechange

fordvolumechanges= stock_data[( 'F', 'Volume')]
fordpercentagevolumechange= fordvolumechanges.pct_change(periods=1)
stock_data['F','%changevolume'] = fordpercentagevolumechange

shellvolumechanges= stock_data[( 'RDS-B', 'Volume')]
shellpercentagevolumechange= shellvolumechanges.pct_change(periods=1)
stock_data['RDS-B','%changevolume'] = shellpercentagevolumechange

bpvolumechanges= stock_data[( 'BP', 'Volume')]
bppercentagevolumechange= bpvolumechanges.pct_change(periods=1)
stock_data['BP','%changevolume'] = bppercentagevolumechange

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Calculating Cumulative Return of stock
stock_data['TSLA','CumulativeRet'] = (1+ stock_data[( 'TSLA', '%changeprice')]).cumprod()

stock_data['PCRFY','CumulativeRet'] = (1+ stock_data[( 'PCRFY', '%changeprice')]).cumprod()

stock_data['GM','CumulativeRet'] = (1+ stock_data[( 'GM', '%changeprice')]).cumprod()

stock_data['F','CumulativeRet'] = (1+ stock_data[( 'F', '%changeprice')]).cumprod()

stock_data['RDS-B','CumulativeRet'] = (1+ stock_data[( 'RDS-B', '%changeprice')]).cumprod()

stock_data['BP','CumulativeRet'] = (1+ stock_data[( 'BP', '%changeprice')]).cumprod()

stock_data = stock_data.sort_index(axis=1)

In [ ]:
# Printing stock data
stock_data

## Step 3: Exploratory Analysis

Let us look at the Adjusted Close price of Tesla and rest of the companies for the last 5 years.

In [ ]:
# Plot the adjusted close price
stock_data[('TSLA','Adj Close')].plot(figsize=(16, 10))

# Define the label for the title of the figure
plt.title("Adjusted Close Price of Tesla", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('Price', fontsize=14)
plt.xlabel('Year', fontsize=14)

# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

# Show the plot
plt.show()

In [ ]:
# Getting adjusted close price
AdjClose=data.iloc[:, stock_data.columns.get_level_values(1)=='Adj Close']

AdjClose.plot(figsize=(16, 10))

# Show the legend
plt.legend()

# Define the label for the title of the figure
plt.title("Adjusted Close Price", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('Price', fontsize=14)
plt.xlabel('Year', fontsize=14)

# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.show()

From the above graph it can be seen that the Adjusted Close price of Tesla was approximately in the same range as rest of the companies that we are comparing until early 2020. Then it started to rise significantly whereas share price of the rest of the companies remained more or less same. 

###### Question 1: Does change in stock prices affect the volume of stock traded of Tesla? What is the relation between volume and close price of Tesla? 

I intend to find that does the change in price of Tesla share have any effect on the volume of shares traded. It is generally expected that more trading of shares will occur when there is a significant change in price as people will rush to take advantage of the price swing.

In [ ]:
# Plot the adjusted close price
stock_data[('TSLA','%changevolume')].plot(figsize=(16, 10))

# Define the label for the title of the figure
plt.title("Daily % change of Volume of Tesla shares traded", fontsize=16)

# Define the labels for x-axis and y-axis
plt.ylabel('% change of Volume', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ticklabel_format(useOffset=False, style='plain',axis='y')
# Plot the grid lines
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

# Show the plot
plt.show()


stock_data[('TSLA','%changeprice')].plot(figsize=(17,10))
plt.suptitle('Daily % change of Tesla. stock price', fontsize=16, color='black')
plt.ylabel('% change of share price', fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)

plt.show() 

The y-axis of both graphs represents the percentage change in the value. This is needed because the daily volume of shares traded will be in the range of hundreds of thousands, and the daily price movement is normally a few dollars. We may compare the two datasets on a similar scale by converting to percentage change. There is no visible association between the number of shares exchanged and the daily price change, as seen in the above graph.

In [ ]:
# Scatter plot between the values
plt.figure(figsize=(16,10))
plt.scatter(stock_data[('TSLA','%changevolume')],stock_data[('TSLA','%changeprice')])
plt.suptitle('Scatter plot between %change in price and volume traded of Tesla stock', fontsize=16, color='black')
plt.ylabel('% change in price of stock', fontsize=14)
plt.xlabel('% change in Volume Traded', fontsize=14)
plt.grid(which="major", color='k', linestyle='-.', linewidth=0.5)
plt.ticklabel_format(useOffset=False, style='plain')

Just from the above scatter plot, it doesn't look like there is much relationship between share price and volume traded of Tesla stock. 

In [ ]:
# Linear regression model
a= stock_data[('TSLA','%changevolume')]
b= stock_data[('TSLA','%changeprice')]
x=np.array(a[1:1258]).reshape((-1,1))
y=b[1:1258]
model = LinearRegression()
model.fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)

The very low value of coefficient of determination tells that there is no relation between the variables. With all the tests done we have reached the same conclusion that there is no relation between change in share price and volume traded of Tesla, which is quite surprising as we would expect that there would be more trading activity when there is a drastic change in price as people will try to benefit from the said change in price.